In [2]:
!pip install vaderSentiment

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 300)
import praw #reddit data api
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
import re #regex
import requests
import json
import csv
import time
import datetime

# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [5]:
#function to get data from pushshift api
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

#get relevant data from data extracted using previous function
def collectSubData(subm):
    subData = [subm['id'], subm['title'], subm['url'], datetime.datetime.fromtimestamp(subm['created_utc']).date()]
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"
    subData.append(flair)
    subStats.append(subData)

In [10]:

#Subreddit to query
sub='Cryptocurrency'
#before and after dates
before = "1630738800" #sept 4 2021
after = "1599202800" #sept 4 2020
#query string
query = "Bitcoin"
subCount = 0
subStats = []

In [11]:
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub)

https://api.pushshift.io/reddit/search/submission/?title=Bitcoin&size=1000&after=1599202800&before=1630738800&subreddit=Cryptocurrency
100
2020-09-08 17:10:56
https://api.pushshift.io/reddit/search/submission/?title=Bitcoin&size=1000&after=1599610256&before=1630738800&subreddit=Cryptocurrency
100
2020-09-13 09:32:20
https://api.pushshift.io/reddit/search/submission/?title=Bitcoin&size=1000&after=1600014740&before=1630738800&subreddit=Cryptocurrency
100
2020-09-17 15:50:20
https://api.pushshift.io/reddit/search/submission/?title=Bitcoin&size=1000&after=1600383020&before=1630738800&subreddit=Cryptocurrency
100
2020-09-22 16:37:14
https://api.pushshift.io/reddit/search/submission/?title=Bitcoin&size=1000&after=1600817834&before=1630738800&subreddit=Cryptocurrency
100
2020-09-27 13:30:12
https://api.pushshift.io/reddit/search/submission/?title=Bitcoin&size=1000&after=1601238612&before=1630738800&subreddit=Cryptocurrency
100
2020-09-29 22:13:42
https://api.pushshift.io/reddit/search/submiss

In [12]:
#organize data into dataframe
data={}
ids=[]
titles=[]
urls=[]
dates=[]
flairs=[]
for stat in subStats:
    ids.append(stat[0])
    titles.append(stat[1])
    urls.append(stat[2])
    dates.append(stat[3])
    flairs.append(stat[4])
data['id']=ids
data['title']=titles
data['url']=urls
data['date']=dates
data['flair']=flairs
df_1=pd.DataFrame(data)
df_1=df_1[df_1['flair']=='Daily Discussion']

In [13]:
# Set Reddit API Credentials
reddit = praw.Reddit(
     client_id="3gRvZgbUzCz_Tg",
     client_secret="BuNkAnAhJFZhtDMw303NyxiIonIkpg",
     user_agent="image-scraper"
)

In [14]:
#collect comments using praw
comments_by_day=[]
for url in df_1['url'].tolist():
    try:
        submission = reddit.submission(url=url)
        submission.comments.replace_more(limit=0)
        comments=list([(comment.body) for comment in submission.comments])
    except:
        comments=None
    comments_by_day.append(comments)

In [15]:
## run vader sentiment analyzer

analyser = SentimentIntensityAnalyzer()

scores=[]
for comments in comments_by_day:
    sentiment_score=0
    try:
        for comment in comments:
            sentiment_score=sentiment_score+analyser.polarity_scores(comment)['compound']
    except TypeError:
        sentiment_score=0
    
    scores.append(sentiment_score)
    
df_1['sentiment score']=scores

In [16]:
df_1.head()

,id,title,url,date,flair,sentiment score


In [ ]:

## merge with spy price and plot

spy=ffn.get('spy', start='2010-01-01')
spy_vals=[]
for date in df_1['date'].tolist():
    try:
        spy_vals.append(float(spy.loc[date]))
    except KeyError:
        spy_vals.append(None)
        
df_1['spy']=spy_vals

In [ ]:
df_1=df_1[['date','sentiment score','spy']]
df_1=df_1.set_index('date')
df_1=df_1[df_1['spy'].notna()]

In [ ]:
df_1.to_csv('positive+negative_sentiment_data_1.csv')